<a href="https://colab.research.google.com/github/Peritec-Corp/Jetson/blob/main/Jetson_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ
- 下記サイトでSDカードを用意

https://qiita.com/c60evaporator/items/a35e811edb49484c53bc

wifi がなさそうなので一旦有線でネットワークに接続

アプリをアップデートすると、nvidia-l4t-bootloader にエラーがでたので
下を対応

https://forums.developer.nvidia.com/t/solution-dpkg-error-processing-package-nvidia-l4t-bootloader-configure/208627

- Python環境を用意
```
sudo apt-get install python3-pip
pip3 install -U pip
pip3 install pyserial
```


# GPS接続

- GT-902PMGG (https://akizukidenshi.com/download/ds/yic/GT-902PMGG_20200717.pdf) \
・ワイヤ：赤色=VCC、黒色=GND、橙色=TXD、緑色=RXD、茶色=1pps\
・赤を3V3、黒をGND、橙色をRXD、緑色をTXDにつないだ\
・茶色は未使用

- GMS7-CR6 (http://akizukidenshi.com/download/ds/canmore/GMS6-CR6_v1.1.pdf) \
一番内側が一番ピン\
・1 Ground Power and signal ground\
・2 Power 3.3V ~ 6.0V DC input\
・3 Serial Data In 2 Asynchronous serial input at RS-232 level, to input command message\
・4 Serial Data Out 2 Asynchronous serial output at RS-232 level, to output NMEA message\
・5 Serial Data In 1 Asynchronous serial input at TTL level, to input command message\
・6 Serial Data Out 1 Asynchronous serial output at TTL level, to output NMEA message\
・1番ピンをGND、2番ピンを5V、5番ピンをRXD、6番ピンをTXDにつないだ。\
・3、4番ピンをショートさせた（こうしないと値が取れなかった、理由は不明）


# 座標取得

以下のプログラムを参考にGPS.pyを作成\
https://github.com/JetsonHacksNano/UARTDemo/blob/master/uart_example.py \
(dev/ttyTHS1 の箇所はもしかしたら違う値を記入する必要があるかもしれない\
 dmesg | grep tty* コマンドで表示される tty\* のどれかだと思うので、いろいろ試す)

そのまま動かすとエラーが発生したので sudo 権限で実行すると値が取れた
```
PermissionError: Permission denied: 'dev/ttyTHS1'
```


# 解析

取得できた値は nmea 形式のデータなのでそれを解析する

データを解析するライブラリがあるのでそれを使う\
https://github.com/inmcm/micropyGPS

```
pip install git+https://github.com/inmcm/micropyGPS.git
```

以下に乗っているプログラムを参考に実行し、動作した\
https://qiita.com/AmbientData/items/fff54c8ac8ec95aeeee6

プログラム
```
import serial
import micropyGPS
import threading
import time

gps = micropyGPS.MicropyGPS(9, 'dd')

def rungps():
	s = serial.Serial(
		port="/dev/ttyTHS1",
		baudrate=9600,
		bytesize=serial.EIGHTBITS,
		parity=serial.PARITY_NONE,
		stopbits=serial.STOPBITS_ONE,                                                       
		)

	s.readline()
	while True:
		if s.inWaiting() > 0:
			try:
				sentence = s.readline()
				sentence = sentence.decode('utf-8')				
				if sentence[0] != '$':
					continue					
				for x in sentence:
					gps.update(x)

			except Exception as exception_error:
				print(str(exception_error))

gpsthread = threading.Thread(target=rungps, args=())
gpsthread.daemon = True
gpsthread.start()

while True:
	if gps.clean_sentences > 20:
		h = gps.timestamp[0] if gps.timestamp[0] < 24 else gps.timestamp[0] - 24
		print(f"{h:2d}:{gps.timestamp[1]:02d}:{gps.timestamp[2]:04.1f}")
		print(f"緯度経度:{gps.latitude[0]:2.8f}, {gps.longitude[0]:2.8f}")
		print(f"海抜：{gps.altitude}")
		print(f"測位利用衛星:{gps.satellites_used}")
		print('衛星番号  (仰角, 方位角, SN比)')
		for k, v in gps.satellite_data.items():
			print(f"{k}: {v}")
		print("")
	time.sleep(3.0)

```


# MQTT

MQTTを通してペリテックサーバーを利用できるようにする\
MQTT 用ライブラリをインストール

```
pip3 install paho-mqtt
```

参考プログラム
```
import random
import time

import paho.mqtt.client as mqtt_client

broker = 'peritec.net'
port = 1883
topic = "/python/mqtt"
client_id = f'python-mqtt-{random.randint(0, 1000)}'
username = 'iot'
password = 'iot-school'

def connect_mqtt():
    def on_connect(client, userdata, flags, rc):
        if rc == 0:
            print("connected to MQTT Broker")
        else:
            print("Failed to connect, return code %d\n", rc)

    client = mqtt_client.Client(client_id)
    client.username_pw_set(username, password)
    client.on_connect = on_connect
    client.connect(broker, port)
    return client

def publish(client):
    msg_count = 0
    while True:
        time.sleep(1)
        msg = f"messages: {msg_count}"
        result = client.publish(topic, msg)
        status = result[0]
        if status == 0:
            print(f"Send `{msg}` to topic `{topic}`")
        else:
            print(f"Failed to send message to topic {topic}")
        msg_count += 1

def run():
    client = connect_mqtt()
    client.loop_start()
    publish(client)

if __name__ == '__main__':
    run()
```

MQTTが利用できた

#MC7700

SIMを使ってLTE回線を通して、通信を行いたい。\
SierraのMC7700を使ってSIMに対応させる。

MC7700 に 3.3V給電、USBでJetsonに接続 (USB接続だと5V給電なので電源だけ別で取る)\
以下コマンドで、USBデバイスとして認識されていることを確認する
```
lsusb
```
これで見た時にMC7700のIDの下側が 68a3 だった場合、モードを切り替える必要がある。

## firmware アップデート
以下でアップデートをする\
（うまくいかないかもしれないがそれならそのまま次へ進む）
https://source.sierrawireless.com/resources/airprime/software/mc7700_ntt_approved_fw3_5_19_00a_qmi/#sthash.Y7w3glch.dpbs \
(SIMはdocomo、モードはqmiでしか動かせなかったので、firmwareはNTTのqmiを選んだ)

## Windows PC用ドライバ
必要に応じてインストール\
(Tagに7700が入っていないがタグが入っている最新だと不明なデバイスが残るからこれを使っておく)\
https://source.sierrawireless.com/resources/airprime/software/airprime-em_mc-series-windows-drivers-qmi-build-5087/#sthash.3FSYhjzY.dpbs

## Jetson用ドライバ
必要に応じてインストール\
(正直うまく入れられたのかがわからない。Jetsonから認識しなかったら頑張って入れる）
https://source.sierrawireless.com/resources/airprime/software/linux-qmi-sdk-software-latest/#sthash.TItbXpzo.dpbs

##モード切替
DirectIP から QMI にモードを変更する必要がある\
(DirectIPでも使えるらしいが自分はうまく動かなかった)\
一旦 WindowsPC につなぎ teraterm等を使用し AT COMMAND を送る
```
AT!ENTERCND="A710"
AT!UDPID?
```

応答が AT!UDPID=68A3 なら以下を入力し、値が変わることを確認する。
```
AT!UDPID=68A2
```

Windows PCにつないだ時に、ネットワーク設定に携帯電話が表示されること

またMC7700をJetsonにつないでコンソールからコマンド入力して\
ログの中に ttyUSB* が表示されることを確認
```
dmesg | grep tty
```

## APN設定

AT COMMAND でAPN設定を入力する\
Jetsonから minicom でコマンドを送る\
ttyUSB* のいずれかでつながると思う
```
sudo apt-get install minicom
sudo minicom -D /dev/ttyUSB*
```

接続後、AT を送って、OKが返ってくればコマンドが送れている。
```
AT
```

APNを設定する・
```
AT+CGDCONT=1, "IP", "[APN名]"
```
ユーザー名、パスワードが必要な場合はさらに入力
```
AT$QCPDPP=1, CHAP, "[PASSWORD]", "[USERNAME]"
```

IPアドレスが振られていれば、設定が成功していると思う
```
AT+CGPADDR?
```

ATコマンド資料\
https://source.sierrawireless.com/resources/airprime/minicard/2130617_ac_ap_umts_supported_at_command_reference/#sthash.GXGnyYv0.dpbs \
https://source.sierrawireless.com/resources/airprime/minicard/2130616_ap_mcsl_umts_lte_extended_at_command_ref/#sthash.CtZlP63K.dpbs \

簡単な参考\
https://docs.monogoto.io/getting-started/general-device-configurations/iot-devices/sierra-wireless-mc7700

## Jetson からネットワーク設定
メニューバーのネットワーク設定の Edit Connections...\
を選択し、Mobile Broadband 設定を作成する

その後メニューバーのネットワーク設定から作成した設定を選択するとインターネットにつなぐことができる

これだけでは、再起動ごと手動でつなぐ必要があるので自動化する

ネットワーク設定の名前を確認する
```
nmcli connection show 
```

接続設定を変更する
```
sudo nmcli c modify 'ネットワーク名' connection.autoconnect yes
```